In [131]:
import pandas as pd
import numpy as np
from pathlib import Path    
from tqdm import tqdm
import csv

In [122]:
#get all genes names from one patient to create header of all_data file
header=['Patient_id']
def get_header():
    csv_file='/home/syrine/Projet Myelome/data/900-04_TPM.csv'
    example=pd.read_csv(csv_file)
    patient_id=csv_file.split('_')[0][33:] #get patient id from csv file name
    example[['GeneID', 'Patient '+patient_id]]=example['GeneID\t'+patient_id].str.split("\t", expand = True)
    example=example.drop('GeneID\t'+patient_id, axis=1) #separate the two columns
    genes=example['GeneID'].values.tolist()
    res=header+genes
    res.append('MRD Response')
    return res
    

In [125]:
#Some patients in the csv files do not have their id but their admission number. 
#This dictionary contains the correspondance between the two in order to change the ones with admission number

def Correspondances_interne_id():
    d={}
    with open("nums_internes_cassioppe.tsv") as f:
        for line in f:
            (value, key)=line.split()
            d[key]=value[2:8]
    return d

dict=Correspondances_interne_id()
print(dict)

{'T37488': '001-35', 'T37487': '001-36', 'T37357': '092-04', 'T37304': '034-10', 'T37018': '047-12', 'T36552': '015-13', 'T36515': '094-02', 'T36226': '095-09', 'T36023': '047-11', 'T35854': '001-33', 'T35624': '001-34', 'T35571': '030-07', 'T35329': '001-30', 'T34888': '091-04', 'T34497': '001-21', 'T34413': '015-07', 'T34378': '013-07', 'T34361': '001-28', 'T34283': '038-15', 'T34266': '056-07', 'T34244': '019-20', 'T34213': '038-07', 'T33945': '059-01', 'T33890': '157-02', 'T33853': '038-15', 'T33759': '001-15', 'T33739': '101-03', 'T33598': '034-04', 'T33503': '057-01', 'T33480': '015-10', 'T33344': '034-34', 'T33069': '038-26', 'T32983': '034-13', 'T32941': '095-19', 'T32841': '152-04', 'T32696': '034-20', 'T32352': '030-07', 'T31955': '165-04', 'T31666': '095-12', 'T31450': '001-39', 'T30574': '013-03', 'T30483': '121-11', 'T30385': '006-08', 'T30237': '091-05', 'T30144': '009-18', 'T29932': '017-03', 'T29873': '020-01', 'T29662': '022-13', 'T29366': '038-25', 'T29327': '031-11',

In [126]:
reponses=pd.read_excel('Cassiopée Réponse-MRD.xls')
reponses.head()
def get_patient_response(patient_id):
    for i in range(len(reponses)):
        if (patient_id==reponses['USUBJID'].values[i][-6:]):
            if (reponses['post_consolidation_MRD at 10-5'].values[i]=='POSITIVE'):
                return 1
            else :
                return 0

In [127]:
#Create one big matrix: lines are patients, columns are genes, last column is the label (MRD+ or MRD-)

directory = '/home/syrine/Projet Myelome/data'
pathlist = Path(directory).glob('*.csv')
headerList=get_header()
not_found=0
def get_all_data ():
    file = open("all_data.csv", "w")
    writer = csv.writer(file)
    writer.writerow(headerList)
    patient_number=0
    for path in tqdm(pathlist):
        res=[]
        csv_file=str(path)
        data=pd.read_csv(csv_file)
        patient_id=csv_file.split('_')[0][33:]
        genes_count=data['GeneID\t'+patient_id].str.split("\t", expand = True)[1].values
        if patient_id[0]=='T':
            if (patient_id not in dict.keys()):
                print(patient_id, ' not found')
            else :
                patient_id=dict[patient_id]
        y=get_patient_response(patient_id)
        res.append(patient_id)
        res=res+genes_count.tolist()
        res.append(y)
        writer.writerow(res)    
    file.close()

In [128]:
get_all_data()

523it [01:31,  5.72it/s]


In [129]:
all_data = pd.read_csv('all_data.csv')

In [130]:
all_data.columns

Index(['Patient_id', 'ENSG00000223972', 'ENSG00000227232', 'ENSG00000243485',
       'ENSG00000237613', 'ENSG00000268020', 'ENSG00000240361',
       'ENSG00000186092', 'ENSG00000238009', 'ENSG00000239945',
       ...
       'ENSG00000266316', 'ENSG00000264728', 'ENSG00000238667',
       'ENSG00000238477', 'ENSG00000271726', 'ENSG00000215616',
       'ENSG00000215611', 'ENSG00000265557', 'ENSG00000238643',
       'MRD Response'],
      dtype='object', length=57907)

In [16]:
# Function to return the constant value columns of a given DataFrame
def remove_constant_value_features(df):
    return [e for e in df.columns if df[e].nunique() == 1]

drop_col = remove_constant_value_features(all_data)
all_data=all_data.drop(drop_col, axis=1)
